In [1]:
import numpy as np
import pandas as pd
import os
import time
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import mlflow


In [34]:
# Répertoire contenant les fichiers CSV
directory = 'data/defi_ai_night_mlops-main/'

# Liste pour stocker les DataFrames individuels
dfs = []

# Parcourir tous les fichiers dans le répertoire
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Chemin complet du fichier
        file_path = os.path.join(directory, filename)
        # Charger le fichier CSV dans un DataFrame
        df = pd.read_csv(file_path)
        # Ajouter le DataFrame à la liste
        dfs.append(df)

# Concaténer tous les DataFrames en un seul DataFrame
data = pd.concat(dfs, ignore_index=True)
data.to_csv('concatenated_reviews.csv', index=False)

# Afficher le DataFrame combiné
data.shape


'# Répertoire contenant les fichiers CSV\ndirectory = \'data/defi_ai_night_mlops-main/\'\n\n# Liste pour stocker les DataFrames individuels\ndfs = []\n\n# Parcourir tous les fichiers dans le répertoire\nfor filename in os.listdir(directory):\n    if filename.endswith(".csv"):\n        # Chemin complet du fichier\n        file_path = os.path.join(directory, filename)\n        # Charger le fichier CSV dans un DataFrame\n        df = pd.read_csv(file_path)\n        # Ajouter le DataFrame à la liste\n        dfs.append(df)\n\n# Concaténer tous les DataFrames en un seul DataFrame\ndata = pd.concat(dfs, ignore_index=True)\ndata.to_csv(\'concatenated_reviews.csv\', index=False)\n\n# Afficher le DataFrame combiné\ndata.shape'

In [36]:
data = pd.read_csv('concatenated_reviews.csv')

In [37]:
data.head()

,DFA_channel1,Fisher_Information_channel1,HFD_channel1,PFD_channel1,SVD_Entropy_channel1,variance_channel1,std_deviation_channel1,mean_channel1,fft_variance_channel1,fft_std_deviation_channel1,...,SVD_Entropy_channel2,variance_channel2,std_deviation_channel2,mean_channel2,fft_variance_channel2,fft_std_deviation_channel2,fft2_variance_channel2,zero_crossing_rate_channel2,complexity_channel2,label
0,-2.340418,-2.444332,1.919123,2.004789,0.059164,0.067505,0.661914,0.652315,1.919123,2.004789,...,6.283398e-07,0.001094,0.000793,0.000003,6.744717e-07,52.0,56.0,0.001083,0.000779,1.0
1,-2.370066,-2.457359,1.933954,2.072673,0.140508,0.108550,0.632009,0.637723,1.933954,2.072673,...,4.043640e-07,0.001116,0.000636,0.000002,5.726524e-07,50.0,62.0,0.001167,0.000405,1.0
2,-2.417069,-2.342087,1.616646,1.613469,0.057907,0.059816,0.658532,0.652315,1.616646,1.613469,...,3.947062e-06,0.001636,0.001987,0.000010,2.349343e-06,22.0,34.0,0.003095,0.006070,1.0
3,-2.345681,-2.260865,2.074892,1.905959,0.279197,0.228970,0.614835,0.622714,2.074892,1.905959,...,3.065704e-06,0.001425,0.001751,0.000007,1.835025e-06,80.0,46.0,0.002409,0.003668,1.0
4,-2.414445,-2.275812,1.825687,1.826842,0.226320,0.141556,0.622150,0.636237,1.825687,1.826842,...,2.409784e-06,0.001751,0.001552,0.000008,2.231318e-06,72.0,44.0,0.002179,0.002717,1.0


In [38]:
data.label.value_counts()

label
0.0    6912
1.0     792
Name: count, dtype: int64

In [39]:
X = data.drop(columns='label')
Y = data['label']

In [40]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=Y)

In [45]:
mlflow.set_experiment("Epileptic Seizure Detection")
mlflow.set_tracking_uri("sqlite:///mlflow_1.db")


<Experiment: artifact_location='file:///c:/Users/khaled/Documents/Compitition%20data/mlruns/285062207815472122', creation_time=1711828159901, experiment_id='285062207815472122', last_update_time=1711828159901, lifecycle_stage='active', name='Epileptic Seizure Detection', tags={}>

## Définition des pipelines

Nous définissons plusieurs pipelines de classification, chacun utilisant le sur-échantillonnage des données avec SMOTE et un algorithme de classification spécifique.

### ExtraTrees (rf)

- Sur-échantillonnage avec SMOTE
- Classificateur : ExtraTreesClassifier

### Régression Logistique (lr)

- Sur-échantillonnage avec SMOTE
- Classificateur : LogisticRegression

### K-Nearest Neighbors (knn)

- Sur-échantillonnage avec SMOTE
- Classificateur : KNeighborsClassifier

### LightGBM (lgbm)

- Sur-échantillonnage avec SMOTE
- Classificateur : LGBMClassifier

### XGBoost (XGB)

- Sur-échantillonnage avec SMOTE
- Classificateur : XGBClassifier

Ces pipelines seront utilisés pour entraîner et évaluer les modèles de classification sur nos données.

In [46]:
# Define a memory object to cache intermediate results
cachedir = '.cache'
memory = Memory(location=cachedir, verbose=0)

pipelines = {
    'rf': ImbPipeline([
        ('sampling', SMOTE()),
        ('classifier', ExtraTreesClassifier())
    ], memory=memory),
     'lr': ImbPipeline([
        ('sampling', SMOTE()),
        ('classifier', LogisticRegression())
    ], memory=memory),
    'knn': ImbPipeline([
        ('sampling', SMOTE()),
        ('classifier', KNeighborsClassifier())
    ], memory=memory),
       'lgbm': ImbPipeline([
        ('sampling', SMOTE()),
        ('classifier', LGBMClassifier())
    ], memory=memory),
       'XGB': ImbPipeline([
        ('sampling', SMOTE()),
        ('classifier', XGBClassifier())
    ], memory=memory),
}


# Grilles de Paramètres

Nous définissons ici les grilles de paramètres pour la recherche par grille des hyperparamètres pour chaque modèle de classification.

## Random Forest (rf)

- **sampling**: Différentes méthodes d'échantillonnage (SMOTE, RandomOverSampler, RandomUnderSampler)
- **classifier__n_estimators**: Nombre d'estimateurs dans Random Forest
- **classifier__max_depth**: Profondeur maximale de chaque arbre dans Random Forest

## Régression Logistique (lr)

- **sampling**: Différentes méthodes d'échantillonnage (SMOTE, RandomOverSampler, RandomUnderSampler)
- **classifier__penalty**: Type de régularisation dans la régression logistique (l1, l2)
- **classifier__C**: Paramètre de régularisation dans la régression logistique

## XGBoost (XGB)

- **sampling**: Différentes méthodes d'échantillonnage (SMOTE, RandomOverSampler, RandomUnderSampler)
- **classifier__learning_rate**: Taux d'apprentissage dans XGBoost
- **classifier__max_depth**: Profondeur maximale de chaque arbre dans XGBoost
- **classifier__n_estimators**: Nombre d'estimateurs dans XGBoost

## K-Nearest Neighbors (knn)

- **sampling**: Différentes méthodes d'échantillonnage (SMOTE, RandomOverSampler, RandomUnderSampler)
- **classifier__n_neighbors**: Nombre de voisins à considérer dans KNN
- **classifier__weights**: Méthode de pondération des voisins dans KNN

## LightGBM (lgbm)

- **sampling**: Différentes méthodes d'échantillonnage (SMOTE, RandomOverSampler, RandomUnderSampler)
- **classifier__num_leaves**: Nombre de feuilles dans LightGBM
- **classifier__learning_rate**: Taux d'apprentissage dans LightGBM
- **classifier__n_estimators**: Nombre d'estimateurs dans LightGBM

Ces grilles de paramètres seront utilisées pour rechercher les meilleures combinaisons d'hyperparamètres pour chaque modèle de classification.


In [47]:
param_grids = {
    'rf': {
        
        'sampling': [SMOTE(), RandomOverSampler(), RandomUnderSampler()],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20]
    },
    'lr': {
        'sampling': [SMOTE(), RandomOverSampler(), RandomUnderSampler()],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': [0.1, 1.0, 10.0]
    },
    'XGB': {
        'sampling': [SMOTE(), RandomOverSampler(), RandomUnderSampler()],
        'classifier__learning_rate': [0.1, 0.01],
        'classifier__max_depth': [3, 5, 7],
        'classifier__n_estimators': [100, 200, 300]
    },
    'knn': {
        'sampling': [SMOTE(), RandomOverSampler(), RandomUnderSampler()],
        'classifier__n_neighbors': [3, 5, 7],
        'classifier__weights': ['uniform', 'distance']
    },
    'lgbm': {
        'sampling': [SMOTE(), RandomOverSampler(), RandomUnderSampler()],
        'classifier__num_leaves': [31, 50, 100],
        'classifier__learning_rate': [0.1, 0.01],
        'classifier__n_estimators': [100, 200, 300]
    }
}


In [ ]:
import warnings

warnings.filterwarnings('ignore')

# Recherche des Meilleurs Modèles avec MLflow

### Enregistrement Automatique de Tous les Essais d'Expérience avec MLFlow

Dans cette section, nous exécutons la recherche par grille pour trouver les meilleurs hyperparamètres pour chaque algorithme de classification. Nous utilisons la validation croisée à 5 plis pour évaluer les performances des modèles.

Nous utilisons également MLflow pour suivre les expériences. Les métriques telles que la précision sur l'ensemble d'entraînement et de test sont enregistrées, ainsi que le meilleur modèle trouvé .

Les performances des modèles seront analysées dans la section suivante.


In [54]:
# Dictionnaire pour stocker les meilleurs modèles
best_models = {}

# Exécution du pipeline pour chaque algorithme
for algo in pipelines.keys():
    print("*"*10, algo, "*"*10)
    # Création de l'objet GridSearchCV pour la recherche par grille des hyperparamètres
    grid_search = GridSearchCV(estimator=pipelines[algo], 
                               param_grid=param_grids[algo], 
                               cv=5, 
                               scoring='accuracy', 
                               return_train_score=True,
                              )
    
    # Lancement de l'enregistrement MLflow pour le suivi des expériences
    mlflow.sklearn.autolog(max_tuning_runs=None)
    
    # Début de la session MLflow
    with mlflow.start_run() as run:
        # Exécution de la recherche par grille
        %time grid_search.fit(X_train, y_train)
        
        # Enregistrement des métriques dans MLflow
        mlflow.log_metric("Train Accuracy", grid_search.best_score_)
        mlflow.log_metric("Test Accuracy", grid_search.score(X_test, y_test))
        mlflow.sklearn.log_model(grid_search.best_estimator_, "best_model")
        
    # Affichage du score d'entraînement et du score de test du meilleur modèle trouvé
    print('Train Score: ', grid_search.best_score_)
    print('Test Score: ', grid_search.score(X_test, y_test))
    
    # Stockage du meilleur modèle dans le dictionnaire
    best_models[algo] = grid_search.best_estimator_
    print()


********** rf **********


2024/03/30 23:27:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


CPU times: total: 1min 35s
Wall time: 1min 38s
Train Score:  0.9639780231933516
Test Score:  0.9656067488643738

********** lr **********


2024/03/30 23:29:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:


CPU times: total: 13.1 s
Wall time: 17.3 s
Train Score:  0.8219999789342853
Test Score:  0.8157040882543802

********** knn **********


...

CPU times: total: 1min 29s
Wall time: 38.4 s
Train Score:  0.8142156707850138
Test Score:  0.8215444516547696

********** lgbm **********


2024/03/30 23:30:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


[LightGBM] [Info] Number of positive: 4423, number of negative: 4423
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6628
[LightGBM] [Info] Number of data points in the train set: 8846, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4423, number of negative: 4423
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000878 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6627
[LightGBM] [Info] Number of data points in the train set: 8846, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4423, number of negative: 4423
[LightGBM] [Info] Auto-choosing col-wise multi-threading, t

2024/03/30 23:33:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


CPU times: total: 15min 44s
Wall time: 2min 54s
Train Score:  0.9641400975342581
Test Score:  0.9630110317975341



In [ ]:
for name, model in best_models.items():
    print(f"{name}")
    print(f"{model}")
    print()

In [ ]:
for name, model in best_models.items():
    print("*"*10, name, "*"*10)
    
    joblib.dump(model, f'best_models/{name}.pkl')
    model = joblib.load(f'best_models/{name}.pkl')
    
    %time y_test_pred = model.predict(X_test_clean)
    print("Test Score (F1)", metrics.f1_score(y_test, y_test_pred))
    
    print("Model Size:", os.path.getsize(f'best_models/{name}.pkl'), "Bytes")